In [4]:
import numpy as np
import os
import glob
import numpy as np
def read_velodyne_bin(bin_path):
    """
    Đọc file .bin từ KITTI và trả về mảng (N, 4): x, y, z, reflectance
    """
    return np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)


In [7]:
def read_kitti_label(label_folder):
    """
    Đọc file .txt nhãn từ KITTI object detection.
    Trả về danh sách bounding box + class:
    [class, x, y, z, h, w, l, ry]
    """
    all_labels  = {}
    for file in sorted(os.listdir(label_folder)):
        if not file.endswith('.txt'):
                    continue
        file_id=os.path.splitext(file)[0]
        label_path=os.path.join(label_folder, file)
        boxes = []
        with open(label_path, 'r') as f:
            for line in f:
                if line.strip() == '':
                    continue
                parts = line.strip().split(' ')
                cls = parts[0]
                if cls == 'DontCare':
                    continue
                # Extract 3D box info
                h, w, l = map(float, parts[8:11])
                x, y, z = map(float, parts[11:14])
                ry = float(parts[14])
                boxes.append({
                    'class': cls,
                    'center': [x, y, z],
                    'size': [l, w, h],
                    'rotation': ry
                })
        all_labels[file_id] = boxes
    return all_labels


In [8]:
all_labels=read_kitti_label(r"E:\WorkSpace\Python\Point_cloud\data\archive\training\label_2_subset" )

In [23]:
import struct
import numpy as np
import open3d as o3d
import sys


def bin_to_pcd(binFileName):
    size_float = 4
    list_pcd = []
    with open(binFileName, "rb") as f:
        byte = f.read(size_float * 4)
        while byte:
            x, y, z, intensity = struct.unpack("ffff", byte)
            list_pcd.append([x, y, z])
            byte = f.read(size_float * 4)
    np_pcd = np.asarray(list_pcd)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np_pcd)
    return pcd



if __name__ == "__main__":
    binFileName = r"E:\WorkSpace\Python\Point_cloud\data\archive\training\velodyne_subset\000000.bin"
    pcd = bin_to_pcd(binFileName)
    o3d.visualization.draw([pcd])
    print(pcd)

[Open3D INFO] Window window_0 created.
[Open3D INFO] ICE servers: ["stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"]
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 

In [ ]:


# ==== Cắt point cloud theo bbox ====
def extract_objects_from_pointcloud(points, bboxes, class_map):
    objects = []
    for box in bboxes:
        cls = box['class']
        if cls not in class_map:
            continue
        center, size = box['center'], box['size']
        l, w, h = size
        x, y, z = center

        # Hộp trục song song (AABB)
        mask = (
            (points[:, 0] > x - l/2) & (points[:, 0] < x + l/2) &
            (points[:, 1] > y - w/2) & (points[:, 1] < y + w/2) &
            (points[:, 2] > z - h/2) & (points[:, 2] < z + h/2)
        )
        pc_object = points[mask][:, :3]
        if len(pc_object) >= 30:
            label_id = class_map[cls]
            objects.append((pc_object, label_id))
    return objects

# ==== Hàm chính để duyệt cả folder ====
def extract_all_objects(velodyne_dir, label_dir, class_map):
    all_objects = []  # List chứa (pc_object, class_id)
    bin_files = sorted(glob.glob(os.path.join(velodyne_dir, "*.bin")))

    for bin_path in bin_files:
        file_id = os.path.splitext(os.path.basename(bin_path))[0]  # '000012'

        label_path = os.path.join(label_dir, f"{file_id}.txt")
        if not os.path.exists(label_path):
            print(f"[!] Thiếu label cho {file_id}, bỏ qua")
            continue

        # Đọc dữ liệu
        points = read_velodyne_bin(bin_path)
        bboxes = read_kitti_label(label_path)
        objects = extract_objects_from_pointcloud(points, bboxes, class_map)

        all_objects.extend(objects)
    return all_objects

In [13]:
class_map = {
    "Car": 0,
    "Pedestrian": 1,
    "Cyclist": 2
}

velodyne_dir = "E:\\WorkSpace\\Python\\Point_cloud\\data\\archive\\training\\velodyne_subset\\"
label_dir = "E:\\WorkSpace\\Python\\Point_cloud\\data\\archive\\training\\label_2_subset\\" 

all_objs = extract_all_objects(velodyne_dir, label_dir, class_map)

print(f"Tổng số object trích ra: {len(all_objs)}")
# all_objs = [(pc1, label1), (pc2, label2), ...]

Tổng số object trích ra: 0
